# 1. Install Dependencies and Setup

In [ ]:
print("Hello, world!")

!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 opencv-python matplotlib
!pip install --force-reinstall protobuf==3.20.0
!pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
imgsize = 1024

# 2. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data2' 

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                #os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

tf.keras.utils.image_dataset_from_directory??

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir, image_size=(imgsize, imgsize))

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
batch[0].shape

In [ ]:
batch[0]

In [ ]:
batch[1]

In [ ]:
batch[0].max()

In [ ]:
batch[0][4].shape

In [ ]:
#plt.imshow(cv2.cvtColor(batch[0][4]/255, cv2.COLOR_BGR2RGB))
plt.imshow(batch[0][4].astype(int))
plt.show()

In [ ]:
batch[0][4]

In [ ]:
cv2.cvtColor(batch[0][4]/255, cv2.COLOR_BGR2RGB)

In [ ]:
img_test = cv2.imread('data2/FAPT/109-010-602.jpg')

In [ ]:
img_test.shape

In [ ]:
img_test

In [ ]:
plt.imshow(cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB).astype(int))
#plt.imshow(img_test.astype(int))
plt.show()

In [ ]:
figx, axx = plt.subplots(ncols=1, figsize=(20,20))
#axx.imshow(img_test.astype(int))
axx.imshow(cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB))

In [ ]:
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
new_batch = data.as_numpy_iterator().next()

In [ ]:
new_batch

In [ ]:
new_batch[0][31]

In [ ]:
#img0 = cv2.imread('test/897.jpg')
#newimg = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)

plt.imshow(cv2.cvtColor(new_batch[0][31], cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
new_batch[0].max()

# 5. Split Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)+1
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size

In [ ]:
val_size

In [ ]:
test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 6. Build Deep Learning Model

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (7,7), 1, activation='relu', input_shape=(imgsize,imgsize,3)))
model.add(MaxPooling2D())

model.add(Conv2D(64, (5,5), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(256, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# 7. Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Test

import cv2

In [ ]:
#img0 = cv2.imread('test/897.jpg')
img0 = cv2.cvtColor(cv2.imread('test/897.jpg'), cv2.COLOR_BGR2RGB)
r1 = img0[:,:,0] # get blue channel
g1 = img0[:,:,1] # get green channel
b1 = img0[:,:,2] # get red channel

plt.imshow(img0)
plt.show()
plt.imshow(r1)
plt.show()
plt.imshow(g1)
plt.show()
plt.imshow(b1)
plt.show()
img1 = cv2.imread('test/1253.jpg')
img2 = cv2.imread('test/1254.jpg')
img3 = cv2.imread('test/1305.jpg')
img4 = cv2.imread('test/1470.jpg')

img5 = cv2.imread('test/1547.jpg')
img6 = cv2.imread('test/1430.jpg')
img7 = cv2.imread('test/1431.jpg')
img8 = cv2.imread('test/1466.jpg')
img9 = cv2.imread('test/1515.jpg')
#plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#plt.show()

In [ ]:
np.expand_dims(img0/255, 0)

In [ ]:
resize0 = tf.image.resize(img0, (imgsize,imgsize))
resize1 = tf.image.resize(img1, (imgsize,imgsize))
resize2 = tf.image.resize(img2, (imgsize,imgsize))
resize3 = tf.image.resize(img3, (imgsize,imgsize))
resize4 = tf.image.resize(img4, (imgsize,imgsize))

resize5 = tf.image.resize(img5, (imgsize,imgsize))
resize6 = tf.image.resize(img6, (imgsize,imgsize))
resize7 = tf.image.resize(img7, (imgsize,imgsize))
resize8 = tf.image.resize(img8, (imgsize,imgsize))
resize9 = tf.image.resize(img9, (imgsize,imgsize))
plt.imshow(resize0.numpy().astype(int))
plt.show()

In [ ]:
#yhat0 = model.predict(np.expand_dims(resize0/255, 0))
yhat0 = model.predict(np.expand_dims(resize0/255, 0))[0][0]
yhat1 = model.predict(np.expand_dims(resize1/255, 0))[0][0]
yhat2 = model.predict(np.expand_dims(resize2/255, 0))[0][0]
yhat3 = model.predict(np.expand_dims(resize3/255, 0))[0][0]
yhat4 = model.predict(np.expand_dims(resize4/255, 0))[0][0]

yhat5 = model.predict(np.expand_dims(resize5/255, 0))[0][0]
yhat6 = model.predict(np.expand_dims(resize6/255, 0))[0][0]
yhat7 = model.predict(np.expand_dims(resize7/255, 0))[0][0]
yhat8 = model.predict(np.expand_dims(resize8/255, 0))[0][0]
yhat9 = model.predict(np.expand_dims(resize9/255, 0))[0][0]

In [ ]:
# should be 1
yhat0

In [ ]:
# should be 1
yhat1

In [ ]:
# should be 1
yhat2

In [ ]:
# should be 1
yhat3

In [ ]:
# should be 1
yhat4

In [ ]:
# should be 0
yhat5

In [ ]:
# should be 0
yhat6

In [ ]:
# should be 0
yhat7

In [ ]:
# should be 0
yhat8

In [ ]:
# should be 0
yhat9

In [ ]:
if yhat9 > 0.5: 
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
new_model = load_model(os.path.join('models', 'imageclassifier.h5'))

In [ ]:
yhatnew = new_model.predict(np.expand_dims(resize/imgsize, 0))

In [ ]:
yhatnew

# 12. Release Memory

In [ ]:
from keras import backend as kbe
kbe.clear_session()

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()